In [1]:
######## snakemake preamble start (automatically inserted, do not edit) ########
import sys;sys.path.extend(['/home/jk/miniconda3/envs/snakemake/lib/python3.12/site-packages', '/scratch/jk/work/github/sc2ts-paper/arg_postprocessing', '/home/jk/miniconda3/envs/snakemake/bin', '/home/jk/miniconda3/envs/snakemake/lib/python3.12', '/home/jk/miniconda3/envs/snakemake/lib/python3.12/lib-dynload', '/home/jk/miniconda3/envs/snakemake/lib/python3.12/site-packages', '/home/jk/.cache/snakemake/snakemake/source-cache/runtime-cache/tmp0bq0kq_b/file/scratch/jk/work/github/sc2ts-paper/arg_postprocessing/notebooks', '/scratch/jk/work/github/sc2ts-paper/arg_postprocessing/notebooks']);import pickle;from snakemake import script;script.snakemake = pickle.loads(b'\x80\x04\x95"\x04\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c\x19sc2ts_v1_2023-02-21.trees\x94\x8c)sc2ts_v1_2023-02-21_recombinants_lbs.json\x94\x8c\x1csc2ts_v1_2023-02-21_pr.trees\x94e}\x94(\x8c\x06_names\x94}\x94\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh\x12h\x06\x8c\x0eAttributeGuard\x94\x93\x94)\x81\x94}\x94\x8c\x04name\x94h\x12sbh\x13h\x15)\x81\x94}\x94h\x18h\x13sbub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94}\x94(h\x0e}\x94h\x10]\x94(h\x12h\x13eh\x12h\x15)\x81\x94}\x94h\x18h\x12sbh\x13h\x15)\x81\x94}\x94h\x18h\x13sbub\x8c\r_params_store\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94}\x94(h\x0e}\x94h\x10]\x94(h\x12h\x13eh\x12h\x15)\x81\x94}\x94h\x18h\x12sbh\x13h\x15)\x81\x94}\x94h\x18h\x13sbub\x8c\r_params_types\x94}\x94\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x13sc2ts_v1_2023-02-21\x94a}\x94(h\x0e}\x94\x8c\nRRR_PREFIX\x94K\x00N\x86\x94sh\x10]\x94(h\x12h\x13eh\x12h\x15)\x81\x94}\x94h\x18h\x12sbh\x13h\x15)\x81\x94}\x94h\x18h\x13sbh:h7ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c\x04/tmp\x94e}\x94(h\x0e}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uh\x10]\x94(h\x12h\x13eh\x12h\x15)\x81\x94}\x94h\x18h\x12sbh\x13h\x15)\x81\x94}\x94h\x18h\x13sbhIK\x01hKK\x01hMhFub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c9logs/sc2ts_v1_2023-02-21_rewire-recombinants-report.ipynb\x94a}\x94(h\x0e}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh\x10]\x94(h\x12h\x13eh\x12h\x15)\x81\x94}\x94h\x18h\x12sbh\x13h\x15)\x81\x94}\x94h\x18h\x13sbh[hXub\x8c\x06config\x94}\x94\x8c\x04rule\x94\x8c\x1arewire_recombinants_report\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8c@/scratch/jk/work/github/sc2ts-paper/arg_postprocessing/notebooks\x94ub.');del script;from snakemake.logging import logger;from snakemake.script import snakemake;import os; os.chdir(r'/scratch/jk/work/github/sc2ts-paper/arg_postprocessing');
######## snakemake preamble end #########


In [2]:
import sc2ts.inference as si
import json
import pandas as pd
import numpy as np
import tszip

In [3]:
rematch_data = snakemake.input[1]

ts_in = tszip.load(snakemake.input[0])
ts_out =  tszip.load(snakemake.input[2])

records = {}
with open(rematch_data) as f:
    for d in json.load(f):
        r = si.RematchRecombinantsLbsResult.fromdict(d)
        records[r.recombinant] = r

In [4]:
tree_in = ts_in.first()
tree_out = ts_out.first()
data = []
for r in records.values():
   
    lbs = r.long_branch_split
    hmm_match = None
    if lbs is None:
        assert len(r.recomb_match.path) == 1
        hmm_match = r.recomb_match
    elif len(lbs.hmm_match.parents) == 1:
        hmm_match = lbs.hmm_match

    if hmm_match is not None:
        old_descendants = tree_in.num_samples(r.recombinant)
        new_descendants = tree_out.num_samples(r.recombinant)
        assert old_descendants == new_descendants
        data.append({
            "recombinant": r.recombinant,
            "num_descendant_samples": old_descendants,
            "rewire_existing": lbs is None,
            "original_muts": len(r.original_match.mutations),
            "new_muts": len(hmm_match.mutations),
        })
df = pd.DataFrame(data).set_index("recombinant")
df["muts_gained"] = df["new_muts"] - df["original_muts"]
df

,num_descendant_samples,rewire_existing,original_muts,new_muts,muts_gained
recombinant,,,,,
1117218,1,True,0,3,3
1101250,22,True,2,3,1
1386676,1,True,1,4,3
1341863,1,True,1,4,3
1254597,1,True,0,3,3
...,...,...,...,...,...
319313,1,True,3,4,1
332746,3,True,2,2,0
455832,6,False,0,3,3


In [5]:
ts_in.num_trees, ts_out.num_trees

(348, 317)

In [6]:
ts_out.num_mutations - ts_in.num_mutations

182

In [7]:
ts_out.num_nodes - ts_in.num_nodes

16

# Rewire existing

In [8]:
df["rewire_existing"].value_counts()

rewire_existing
True     58
False    16
Name: count, dtype: int64

In [9]:
df_re = df[df["rewire_existing"]]
df_re

,num_descendant_samples,rewire_existing,original_muts,new_muts,muts_gained
recombinant,,,,,
1117218,1,True,0,3,3
1101250,22,True,2,3,1
1386676,1,True,1,4,3
1341863,1,True,1,4,3
1254597,1,True,0,3,3
967486,1,True,3,6,3
1096654,1,True,0,3,3
1028661,1,True,0,3,3
1344253,3,True,1,4,3


In [10]:
df_re["muts_gained"].value_counts()

muts_gained
 3    35
 2    13
 1     7
 0     2
-1     1
Name: count, dtype: int64

In [11]:
df_re["muts_gained"].sum()

np.int64(137)

In [12]:
df_re["num_descendant_samples"].value_counts()

num_descendant_samples
1      35
2      11
3       5
6       2
22      1
13      1
4       1
17      1
653     1
Name: count, dtype: int64

In [13]:
df_re["num_descendant_samples"].median()

np.float64(1.0)

In [14]:
df_re["num_descendant_samples"].sum()

np.int64(793)

# Rewire long branch splits

In [15]:
df_lbs = df[~df["rewire_existing"]]
df_lbs

,num_descendant_samples,rewire_existing,original_muts,new_muts,muts_gained
recombinant,,,,,
999972,1,False,1,4,3
1121428,1,False,0,3,3
1012974,5,False,1,2,1
1067599,1,False,1,3,2
803921,4,False,0,1,1
732807,2,False,3,4,1
822854,283545,False,31,25,-6
874683,1,False,1,4,3
520914,6,False,1,1,0


In [16]:
df_lbs["muts_gained"].value_counts()

muts_gained
 3     7
 1     3
 2     2
 0     2
-6     1
-15    1
Name: count, dtype: int64

## Look at majority first

In [17]:
df_lbs = df_lbs[df_lbs["muts_gained"] >= 0]
df_lbs

,num_descendant_samples,rewire_existing,original_muts,new_muts,muts_gained
recombinant,,,,,
999972,1,False,1,4,3
1121428,1,False,0,3,3
1012974,5,False,1,2,1
1067599,1,False,1,3,2
803921,4,False,0,1,1
732807,2,False,3,4,1
874683,1,False,1,4,3
520914,6,False,1,1,0
833355,1,False,1,3,2


In [18]:
df_lbs["muts_gained"].value_counts()

muts_gained
3    7
1    3
2    2
0    2
Name: count, dtype: int64

In [19]:
df_lbs["muts_gained"].sum()

np.int64(28)

In [20]:
df_lbs["num_descendant_samples"].sum()

np.int64(33)

## Now the outliers

In [21]:
df_lbs = df[~df["rewire_existing"]]
df_lbs = df_lbs[df_lbs["muts_gained"] < 0]
df_lbs

,num_descendant_samples,rewire_existing,original_muts,new_muts,muts_gained
recombinant,,,,,
822854,283545,False,31,25,-6
200039,538257,False,24,9,-15
